In [1]:
from vkprog_data_prep import load_booking_data
import datetime as dt
import pandas as pd

In [7]:
%%time
date_view      = dt.datetime(2018,10,9) # only works for odd calendar weeks!!!

def branchen_data(date_view):
    
    bd = load_booking_data()
    
    kunden_branchen_df  = bd.loc[:,["Endkunde_NR",
                                    "Kampagne_Erfassungsdatum",
                                    "Endkunde_Branchengruppe_ID"] ]

    auftrag_branchen_df = bd.loc[:,["Endkunde_NR",
                                    "Kampagne_Erfassungsdatum",
                                    "Auftrag_Branchengruppe_ID"] ]

    kunden_branchen_df.columns  = ["Endkunde_NR",
                                   "Kampagne_Erfassungsdatum",
                                   "Branchen_ID"]
    
    auftrag_branchen_df.columns = kunden_branchen_df.columns

    branchen_df = (kunden_branchen_df.append(auftrag_branchen_df)
                                     .dropna()
                                     .drop_duplicates() )
    
    # Define filter (boolean pd.Series) based on view_date
    filter_boolean = (bd.loc[:,"Kampagne_Erfassungsdatum"] < date_view)
    
    # Span pivot table with filter:
    branchen_df = (branchen_df
                   .loc[filter_boolean,:]
                   .pivot_table(
                        index      = ["Endkunde_NR"],
                        columns    = ["Branchen_ID"],
                        values     = ["Branchen_ID"],
                        aggfunc    = "count",
                        fill_value = 0) )
    
    #Flatten and rename:
    branchen_df = pd.DataFrame(branchen_df.to_records(index=True))
    new_col_names = [x.replace("('Kampagne_Erfassungsdatum',","B").replace(" '","").replace("')","") for x in branchen_df.columns]
    branchen_df.columns = new_col_names
    
    #Decode into 0/1:
    for branche in branchen_df.columns[1:]:
        branchen_df.loc[:,branche] = branchen_df.loc[:,branche].apply(lambda x: min(x,1))
        
    return branchen_df

Wall time: 0 ns


In [10]:
%%time
damn = branchen_data(date_view=date_view)

2019-10-09 10:39:28 [INFO] Started loading binary file
2019-10-09 10:39:28 [INFO] Reading from file C:\Users\stc\data\vkprog\bd_data.feather
2019-10-09 10:39:29 [INFO] Finished loading binary file in 0.16s (0.78s CPU)


Wall time: 13.8 s


In [4]:
a = [1,2,3,4,5]
b = [1,3,5,6]
list(set(a) & set(b))

[1, 3, 5]

In [17]:
list(set(damn.columns) & set(['B105']))

['B105']

In [3]:
10**10


10000000000

In [5]:
import numpy as np


In [8]:
np.inf + 5

inf

In [9]:
import seaborn as sns
sns.load_dataset("flights")
flights = flights.pivot("month", "year", "passengers")
ax = sns.heatmap(flights)

NameError: name 'flights' is not defined

# Debugging CRM-data prep>

In [1]:
from vkprog_crm_prep import crm_train_scoring

In [2]:
(crm_train_df, crm_score_df) = crm_train_scoring(day=7,
    month=10,
    year_score=2019,
    year_train=2018,
    year_span=4)

2019-10-10 11:59:23 [INFO] Start.
2019-10-10 11:59:23 [INFO] Started loading binary file
2019-10-10 11:59:23 [INFO] Reading from file C:\Users\stc\data\vkprog\crm_data_vkprog.feather
2019-10-10 11:59:23 [INFO] Finished loading binary file in 0.22s (0.36s CPU)
2019-10-10 11:59:34 [INFO] Finished.


In [3]:
crm_train_df.sample(n=5)

,Endkunde_NR,RY_0_Anz_Besprechung,RY_1_Anz_Besprechung,RY_2_Anz_Besprechung,RY_3_Anz_Besprechung,RY_0_Anz_Besuch,RY_1_Anz_Besuch,RY_2_Anz_Besuch,RY_3_Anz_Besuch,RY_0_Anz_Brief_Dankeskarte,RY_1_Anz_Brief_Dankeskarte,RY_2_Anz_Brief_Dankeskarte,RY_3_Anz_Brief_Dankeskarte,RY_0_Anz_E-Mail,RY_1_Anz_E-Mail,RY_2_Anz_E-Mail,RY_3_Anz_E-Mail,RY_0_Anz_Event_Veranstaltung,RY_1_Anz_Event_Veranstaltung,RY_2_Anz_Event_Veranstaltung,RY_3_Anz_Event_Veranstaltung,RY_0_Anz_Telefon,RY_1_Anz_Telefon,RY_2_Anz_Telefon,RY_3_Anz_Telefon,RY_0_Anz_Anderes,RY_1_Anz_Anderes,RY_2_Anz_Anderes,RY_3_Anz_Anderes,Letzter_Kontakt_Delta_Anderes,Letzter_Kontakt_Delta_Besprechung,Letzter_Kontakt_Delta_Besuch,Letzter_Kontakt_Delta_Brief_Dankeskarte,Letzter_Kontakt_Delta_E-Mail,Letzter_Kontakt_Delta_Event_Veranstaltung,Letzter_Kontakt_Delta_Telefon,Letzter_Kontakt_Delta_global
30298,585540,0,0,0,0,0,0,0,0,0,1,0,0,3,3,6,4,0,0,0,0,0,0,0,0,0,0,0,0,inf,inf,inf,608.0,123.0,inf,inf,123.0
43593,638288,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,inf,inf,inf,inf,152.0,inf,inf,152.0
9880,467361,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,inf,inf,inf,inf,975.0,inf,inf,975.0
39483,621974,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,inf,inf,inf,inf,551.0,inf,inf,551.0
10301,470170,0,0,0,0,0,0,0,0,0,0,0,0,3,6,9,9,0,0,0,0,0,0,0,0,0,0,0,0,inf,inf,inf,inf,16.0,inf,inf,16.0


In [4]:
#crm_score_df
desc_col(crm_train_df)

NameError: name 'desc_col' is not defined

In [6]:
debug_df

,Endkunde_NR,RY_0_Anz_Besprechung,RY_1_Anz_Besprechung,RY_2_Anz_Besprechung,RY_3_Anz_Besprechung,RY_0_Anz_Besuch,RY_1_Anz_Besuch,RY_2_Anz_Besuch,RY_3_Anz_Besuch,RY_0_Anz_Brief_Dankeskarte,RY_1_Anz_Brief_Dankeskarte,RY_2_Anz_Brief_Dankeskarte,RY_3_Anz_Brief_Dankeskarte,RY_0_Anz_E-Mail,RY_1_Anz_E-Mail,RY_2_Anz_E-Mail,RY_3_Anz_E-Mail,RY_0_Anz_Event_Veranstaltung,RY_1_Anz_Event_Veranstaltung,RY_2_Anz_Event_Veranstaltung,RY_3_Anz_Event_Veranstaltung,RY_0_Anz_Telefon,RY_1_Anz_Telefon,RY_2_Anz_Telefon,RY_3_Anz_Telefon,RY_0_Anz_Anderes,RY_1_Anz_Anderes,RY_2_Anz_Anderes,RY_3_Anz_Anderes,Letzter_Kontakt_Delta_Anderes,Letzter_Kontakt_Delta_Besprechung,Letzter_Kontakt_Delta_Besuch,Letzter_Kontakt_Delta_Brief_Dankeskarte,Letzter_Kontakt_Delta_E-Mail,Letzter_Kontakt_Delta_Event_Veranstaltung,Letzter_Kontakt_Delta_Telefon,Letzter_Kontakt_Delta_global
0,100034,0,0,0,0,0,0,2,5,0,0,1,0,14,8,9,7,0,0,0,0,0,0,0,0,0,0,0,0,inf,inf,944.0,986.0,26.0,inf,inf,26.0
1,100039,0,0,0,0,1,0,0,0,0,0,0,0,8,8,12,1,0,0,0,0,1,1,0,0,0,0,0,0,inf,inf,144.0,inf,26.0,inf,320.0,26.0
2,100043,0,0,0,0,0,0,0,0,0,0,0,0,0,4,12,9,0,0,0,0,0,0,0,0,0,0,0,0,inf,inf,1719.0,inf,576.0,inf,inf,576.0
3,100045,0,0,0,0,0,1,0,0,2,0,1,0,11,8,9,6,2,0,0,0,0,1,0,0,0,0,0,0,inf,inf,530.0,2.0,26.0,178.0,401.0,2.0
4,100048,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,1,0,0,0,0,0,0,0,0,0,0,0,0,inf,inf,inf,inf,388.0,inf,inf,388.0
5,100059,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,1,0,0,0,0,0,0,0,0,0,0,0,0,inf,inf,inf,inf,27.0,inf,inf,27.0
6,100061,0,0,0,0,0,0,0,0,1,0,0,0,7,4,10,7,0,0,0,0,0,0,0,1,0,0,0,0,inf,inf,inf,146.0,26.0,inf,1277.0,26.0
7,100064,0,0,0,0,2,0,2,1,0,0,0,0,10,13,16,1,0,0,0,0,0,0,3,0,0,0,0,0,inf,inf,192.0,inf,26.0,inf,950.0,26.0
8,100066,0,0,0,0,1,1,0,0,2,0,2,2,57,40,65,27,2,0,0,0,2,10,12,19,0,0,0,0,inf,inf,66.0,2.0,2.0,178.0,249.0,2.0
9,100068,0,0,0,0,0,0,0,2,2,0,2,1,15,40,15,12,2,0,0,0,1,2,1,6,0,0,0,0,inf,inf,1109.0,2.0,26.0,178.0,347.0,2.0
